## Pontos em mapas

Além dos dados _geométricos_ dados pelos arquivos GeoJSON ou TopoJSON, muitas bases de dados incluem informações geográficas em campos de cordenadas `longitude` e `latitute`, ou tomando referência em regiões geográficas, como por exemplo o nome de países, estados, cartões postais, _etc_., que podem ser mapeadas em coordenadas usando um [serviço de geocódigo](https://en.wikipedia.org/wiki/Geocoding). Em alguns casos, o dado é informativo o suficiente para conseguirmos ver padrões apenas traçando-os &mdash; sem precisar de nenhuma projeção de mapa!

In [ ]:
zipcodes = data.zipcodes.url
zipcodes

Podemos ver cada agência dos correios usando uma pequena (1-pixel) marca `square` (quadrada). Todavia, para indicar a posição nós _não_ usamos os canais `x` e `y`. _Qual é o por quê disto?_

Enquando projeções cartográficas de mapeiam coordenadas (`longitude`, `latitude`) em coordenadas (`x`, `y`), elas podem fazê-lo de forma arbitrária. Não tem garantia, por exemplo, que `longitude` → `x` e `latitude` → `y`! Ao invés disso, o Altair inclue os canais de codificação especiais `longitude` e `latitude` para lidar com coordenadas geográficas. Esses canais indicam quais campos de dados devem ser mapeados nas coordenadas `longitude` e `latitude`, e então aplica a projeção para mapear essas coordenadas em posições (`x`, `y`).

In [ ]:
alt.Chart(zipcodes).mark_square(
    size=1, opacity=1
).encode(
    longitude='longitude:Q', # aplique o campo chamado 'longitude no canal longitude'
    latitude='latitude:Q'    # aplique o campo chamado 'latitude no canal latitude'
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

_Traçando apenas códigos postais, podemos ver o contorno dos Estados Unidos e extrair padrões interessantes acerca da densidade de agências de correios, sem um mapa base ou elementos de referência acessórios._

Foi usada a projeção `albersUsa`, que toma algumas liberdades com a geometria da Terra, com versões em escala do Alasca e do Havaí no canto inferior esquerdo. Como não especificamos os parâmetros de projeção`scale` ou `translate`, o Altair automaticamente os configura para serem projetados.

Podemos agora fazer mais perguntas sobre nossa base de dados. Por exemplo, ela tem alguma lógica ou razão para a alocação dos códigos postais? Para responder essa pergunta, podemos adicionar cores nos baseando nos dois primeiros digítos dos códigos postais. Primeiro adicionamos uma `calculate transform` para extrair o primeiro dígito, e então codificar o resultado usando o canal de cor.

In [ ]:
alt.Chart(zipcodes).transform_calculate(
    digit='datum.zip_code[0]'
).mark_square(
    size=2, opacity=1
).encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
    color='digit:N'
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

_Para dar um zoom em um dado em específico, adicione um filter transform para limitar os dados mostrados! Tente adicionar uma [seleção interativa](https://github.com/uwdata/visualization-curriculum/blob/master/altair_interaction.ipynb) para filtrar em um único dígito e dinamicamente atualizar o mapa. E se atente para usar strings (\`'1'\`) ao invés de números (\`1\`) ao filtrar os valores dos dígitos!_

(Esse exemplo é inspirado na clássica visualização [zipdecode](https://benfry.com/zipdecode/), por Ben Fry!)

Talvez nos perguntemos o que a ordem dos códigos postais pode indicar. Uma maneira de explorar issa questão é conectar cada código postal usando uma marca `line`, como feito na visualização [ZipScribble](https://eagereyes.org/zipscribble-maps/united-states), por Robert Kosara.

In [ ]:
alt.Chart(zipcodes).transform_filter(
    '-150 < datum.longitude && 22 < datum.latitude && datum.latitude < 55'
).transform_calculate(
    digit='datum.zip_code[0]'
).mark_line(
    strokeWidth=0.5
).encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
    color='digit:N',
    order='zip_code:O'
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

_Agora podemos ver como os códigos postais se aglomeram em áreas menores, indicando uma alocação hierárquica dos códigos por localização, mas com uma notável variação por entre as aglomerações locais._

Se você estava atento para nossos mapas anteriores, deve ter notado que temos códigos postais sendo marcados no canto superior esquerdo! Esses correspondem a territórios como Porto Rico ou Samoa Americana, que contêm códigos postais americanos, mas são mapeadas como coordenadas `null` (`0`, `0`) pela projção `albersUsa`. Ademais, o Alasca e o Havaí podem complicar nossa visualização por ligação de segmentos. Em resposta, o código acima inclue um filtro adicional que remove pontos fora dos nossos limites de `longitude` e `latitude`.

_Remova os filtros acima e veja o que acontece!_

## Mapeamento de Símbolos

Agora vamos combinar um mapa base e dados a serem traçados como camadas separadas. Vamos examinar a rede americana de voos comerciais, considerando ambos os aeroportos e rotas aéreas. Para fazê-lo, vamos precisar de três bases de dados. Para nosso mapa base, vamos usar um arquivo TopoJSON para os Estados Unidos com uma resolução de 10m, contendo as características para estados (`states`)  ou países (`countries`):

In [ ]:
usa = data.us_10m.url
usa

Para os aeroportos, vamos usar uma base de dados com campos para as coordenadas de `longitude` e `latitude` de cada aeroporto, além do código `iata` de cada aeroporto &mdash; por exemplo, `'SEA'` para o  [Aeroporto Internacional de Seattle-Tacoma](https://en.wikipedia.org/wiki/Seattle%E2%80%93Tacoma_International_Airport).

In [ ]:
airports = data.airports.url
airports

Finalmente, vamos usar uma base de dados que contém os campos `origin` (origem) e `destination` (destino) com os códigos IATA para os aeroportos correspondentes:

In [ ]:
flights = data.flights_airport.url
flights

flights = data.flights_airport.url
flightsamos começar criando um mapa base usando a projeção `albersUsa`, e adicionar uma camada que traça marcas `circle` para cada aeroporto:

In [ ]:
alt.layer(
    alt.Chart(alt.topo_feature(usa, 'states')).mark_geoshape(
        fill='#ddd', stroke='#fff', strokeWidth=1
    ),
    alt.Chart(airports).mark_circle(size=9).encode(
        latitude='latitude:Q',
        longitude='longitude:Q',
        tooltip='iata:N'
    )
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

_Quantos aeroportos! Obviamente, nem todos eles são grandes hubs._

Assim como a nossa base de dados de códigos postais, nossos dados de aeroportos incluem pontos que ficam fora do território continental dos Estados Unidos. Então, novamente podemos ver pontos no canto superior esquerdo. Talvez seja de nossa vontade filtrar esses pontos, mas antes disso, precisamos saber mais sobre eles.

_Atualize a projeção do mapa para `albers` &ndash; evitando o comportamento idiossincrático de `albersUsa` &ndash; para que as localizações verdadeiras desses pontos adicionais sejam mostradas!_

Agora, ao invés de mostrar todos os aeroportos da mesma maneira, vamos identificar os maiores pelo número de rotas que saem de cada aeroporto. Vamos usar a base de dados `routes` (rotas, em português) como nossa principal fonte de dados: ela contém uma lista de rotas aéreas as quais podemos agregar para contar o número de rotas para cada aeroporto de origem (`origin`).

Entretanto, a base de dados `routes` não inclui as _localizações_ dos aeroportos! Para ampliar os dados `routes` com localizações, precisaremos de uma transformação de dados: `lookup` (procurar, em português). A transformação `lookup` tira um valor de uma coluna em uma base de dados inicial e a usa como _chave_ e procura por alguma informação relacionada em outra tabela. Nesse caso, queremos relacionar o aeroporto em `origin` na nossa base de dados `routes` com o campo `iata` dentro de `airports` e extrair as coordenadas correspondentes pelas colunas `latitude` e `longitude`.

In [ ]:
alt.layer(
    alt.Chart(alt.topo_feature(usa, 'states')).mark_geoshape(
        fill='#ddd', stroke='#fff', strokeWidth=1
    ),
    alt.Chart(flights).mark_circle().transform_aggregate(
        groupby=['origin'],
        routes='count()'
    ).transform_lookup(
        lookup='origin',
        from_=alt.LookupData(data=airports, key='iata',
                             fields=['state', 'latitude', 'longitude'])
    ).transform_filter(
        'datum.state !== "PR" && datum.state !== "VI"'
    ).encode(
        latitude='latitude:Q',
        longitude='longitude:Q',
        tooltip=['origin:N', 'routes:Q'],
        size=alt.Size('routes:Q', scale=alt.Scale(range=[0, 1000]), legend=None),
        order=alt.Order('routes:Q', sort='descending')
    )
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

_Qual aeroporto nos EUA tem o maior número de decolagens?_

Agora que podemos ver os aeroportos, podemos querer interagir com eles para entender melhor a estrutura do tráfego aéreo. Podemos adicionar uma camada de marcação `rule` (regra, em português) que representa os caminhos dos aeroportos de origem (`origin`) para os aeroportos de destino (`destination`), isso requer duas transformações `lookup` para recuperar as coordenadas de cada ponto. Ademais, podemus usar a seleção `single` (único, em portugês) para filtrar essas rotas, de forma que apenas voos originados no aeroporto selecionado sejam mostrados.

_Começando pelo mapa estatístico acima, você pode construir uma versão interativa? Sinta-se livre para pular o código abaixo e trabalhar com o mapa interativo primeiro, e pense como você poderia construí-lo por conta própria!_

In [ ]:
#seleção interativa para o aeroporto de origem
#selecione o aeroporto mais próximo do cursor do mouse
origin = alt.selection_single(
    on='mouseover', nearest=True,
    fields=['origin'], empty='none'
)
#
# referência de dados compartilhados para lookup transforms
foreign = alt.LookupData(data=airports, key='iata',
                         fields=['latitude', 'longitude'])
    
alt.layer(
    # mapa base dos Estados Unidos
    alt.Chart(alt.topo_feature(usa, 'states')).mark_geoshape(
        fill='#ddd', stroke='#fff', strokeWidth=1
    ),
    # linhas das rotas saindo do aeroporto de origem e chegando no de destino
    alt.Chart(flights).mark_rule(
        color='#000', opacity=0.35
    ).transform_filter(
        origin # filtra para apenas a origem selecionada
    ).transform_lookup(
        lookup='origin', from_=foreign # origem lat/lon
    ).transform_lookup(
        lookup='destination', from_=foreign, as_=['lat2', 'lon2'] # dest lat/lon
    ).encode(
        latitude='latitude:Q',
        longitude='longitude:Q',
        latitude2='lat2',
        longitude2='lon2',
    ),
    # determinar o tamanho dos aeroportos pelo número de rotas saindo do mesmo
    # 1. conjunto de dados de voos agregados-aeroportos
    # 2. pesquisa de dados de localização do conjunto de dados de aeroportos
    # 3. remove Porto Rico (PR) e Ilhas Virgens (VI)
    alt.Chart(flights).mark_circle().transform_aggregate(
        groupby=['origin'],
        routes='count()'
    ).transform_lookup(
        lookup='origin',
        from_=alt.LookupData(data=airports, key='iata',
                             fields=['state', 'latitude', 'longitude'])
    ).transform_filter(
        'datum.state !== "PR" && datum.state !== "VI"'
    ).add_selection(
        origin
    ).encode(
        latitude='latitude:Q',
        longitude='longitude:Q',
        tooltip=['origin:N', 'routes:Q'],
        size=alt.Size('routes:Q', scale=alt.Scale(range=[0, 1000]), legend=None),
        order=alt.Order('routes:Q', sort='descending') # coloca círculos menores em cima
    )
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

_Passe o mouse sobre o mapa para explorar a rede de voos!_